# MATH 468 - Stochastic Calculus II - HW 4

In [1]:
'''Packages Used'''
import pandas as pd
import numpy as np
import datetime

'''Code Deck Import'''
from StoCalCodeDeck import walk_params

'''Data Import Packages'''
import yfinance as yf

## Problem 1 (American options with N large)

*Note*: This problem is a continuation of Problem 3 in HW #3. The objective is to generalize your code to American options. Please provide the answers to the questions below as well as a sample of your code.

We consider the same market model as in Problem 3 of HW #3. We consider an American put option with strike price $K = \$1,025$ and maturity $T = 1$ year.

(a) Find the price of the option at time $t_0 = 0$ if it is American and compare with the price of the European put found in Problem 1.  
(b) What is the earliest possible time of optimal exercise of the American put?  
(c) Repeat questions (a) and (b) in the case where $r = 0\%$. What do you conclude?  

### Solution to Part (a)

In [2]:
'''Expanding the Function to Include American/European Nature of the Option'''

def build_tree(N, T, mu, sigma, s0, k, r, option_type, nature):
    '''Establish time delta -- each variable represents
    T - time to maturity in years
    N - number of periods (i.e. time steps)'''
    dt = T / N

    '''Establish up and down factos'''
    u = np.exp(mu*dt + sigma*np.sqrt(dt))
    d = np.exp(mu*dt - sigma*np.sqrt(dt))
    
    '''Establish risk neutral probability'''
    q = ((1+r*dt) - d) / (u - d)
    
    '''Constructs the data frame -- each column and its representation
    index - number
    layer - time step
    stock_price - stock price at that node and layer
    phi - shares in replicating protfolio
    psi - amount of funds in money market
    option value - option price at that node'''
    tree = pd.DataFrame(data=None, index=range(0,sum(range(N+2))))
    tree.index.name = 'node'
    tree['layer'] = ""
    tree['stock_price'] = 0
    tree['phi'] = 0
    tree['psi'] = 0
    
    '''inputs intial stock price'''
    tree['stock_price'].loc[0] = s0

    '''Inputs layers (time steps) and further stock prices'''
    node = 0
    for i in range(0,N+2):
        if i < (N+1):
            tree.iloc[range(node,node+i+1),0] = i #Layer
            tree.iloc[range(node,node+i+1),1] = s0 * d ** (np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #Stock Price
            node += i + 1

    '''Puts intrinsic option value in at every node'''
    if option_type == 'c':
        tree['option_value'] = tree['stock_price'].apply(lambda x: x-k if x-k > 0 else 0)
    elif option_type == 'p':
        tree['option_value'] = tree['stock_price'].apply(lambda x: k-x if k-x > 0 else 0)
    
    '''resets node to N-1'''
    node= node - N
    
    '''Calculates option value and replicating portfolio going backwards'''
    for i in reversed(range(N)):
        price_range = (1 / (1+r*dt)) * (q * tree.iloc[range(node,node+i+1),4].to_numpy() + (1-q) * tree.iloc[range(node-1,node+i),4].to_numpy()) # price option
        if nature == 'european':
            tree.iloc[range(node-i-2,node-1),4] = price_range
        elif nature == 'american':
            tree.iloc[range(node-i-2,node-1),4] = np.maximum(price_range, tree.iloc[range(node-i-2,node-1),4].to_numpy())
        tree.iloc[range(node-i-2,node-1),2] = (tree.iloc[range(node,node+i+1),4].to_numpy() - tree.iloc[range(node-1,node+i),4].to_numpy()) / (tree.iloc[range(node,node+i+1),1].to_numpy() - tree.iloc[range(node-1,node+i),1].to_numpy()) # get delta
        tree.iloc[range(node-i-2,node-1),3] = (tree.iloc[range(node,node+i+1),4].to_numpy() - tree.iloc[range(node-i-2,node-1),2].to_numpy() * tree.iloc[range(node,node+i+1),1].to_numpy()) / (1 + r*dt) # Money Market
        node = node - i - 1

    return tree

In [3]:
'''Value the American Put'''
T=1 # 1 Year
N=60 # 60 Periods
mu, sigma = walk_params(u=0.005,d=0.003,T=T,N=N)
tree_a = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0.05, option_type='p', nature='american')
print('the price of the american put is {:.2f}'.format(tree_a.iloc[0,-1]))

C:\Users\jdean\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


the price of the american put is 25.00


In [4]:
tree_e = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0.05, option_type='p', nature='european')
tree_e.iloc[0,:]
print('the price of the european put is {:.2f}'.format(tree_e.iloc[0,-1]))

the price of the european put is 3.57


The price of the European Put is $\$3.59$ while the American is $\$25.00$, which makes sense as the underlying currently trades $\$25$ cheap to the strike.  Therefore, one could buy the put and exercise it immediately to earn $\$25$ in P&L, while the european put would require waiting until expiry.

### Solution to Part (b)

The earliest possible time for early exercise is right now, as shown in the first few time steps in the dataframe below.

In [5]:
tree_a.loc[:5]

,layer,stock_price,phi,psi,option_value
node,,,,,
0,0,1000.000,-1.0,1024.146545,25.000
1,1,997.000,-1.0,1024.146545,28.000
2,1,1005.000,-1.0,1024.146545,20.000
3,2,994.009,-1.0,1024.146545,30.991
4,2,1001.985,-1.0,1024.146545,23.015
5,2,1010.025,-1.0,1024.146545,14.975


### Solution to Part (c)

In [6]:
'''Value the American Put, 0 rates'''
T=1 # 1 Year
N=60 # 60 Periods
mu, sigma = walk_params(u=0.005,d=0.003,T=T,N=N)
tree_a = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0, option_type='p', nature='american')
print('the price of the american put is {:.2f}'.format(tree_a.iloc[0,-1]))

C:\Users\jdean\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


the price of the american put is 28.60


In [7]:
tree_e = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0, option_type='p', nature='european')
tree_e.iloc[0,:]
print('the price of the european put is {:.2f}'.format(tree_e.iloc[0,-1]))

the price of the european put is 28.60


The options trade at the same price as there is no market drift to push the stock price up.  This makes the european put more valuable, and offers more potential for intrinsic value for the american put

## Problem 2 (Option pricing from real data.)

*Note*: This problem is the continuation of Prob. 3 in HW #1. We will use the data to price options and compare with the market prices.

Consider the asset that you selected in HW #1. For this asset, select three call options and one put option. All options should have approximately the same expiration date, say about 3 months in the future. The three calls should have three different strike prices, typically one *in-the-money*, one *at-the-money*, and one *out-of-the-money*. (Consider values relatively close to the current price and, if you have access to this information, for which the trading volume is relatively large.) The put option should be *at-the-money*. Record the current price of these options. Note that all traded options are American options. Also determine the interest rate to be used for your calculations: typically, you can consider the LIBOR rate, or the 10-year US treasury yield.

(a) We assume that the CRR model represents the stock price and that its assumptions are satisfied. Based on historical data (about 3 months to a year in the past), determine the parameters $\mu$ and $\sigma$ for the stock that you chose following the methods discussed in class. (I recommend to download current data, but feel free to use past data if that is all you have.)  
(b) Price the selected put option using an $N$-period binomial model for different values of $N$ (which you can choose on your own). As a hint, you can for instance start with $N = 2$ or $N = 3$, which should be approximately monthly-periods. You can then use approximately weekly periods ($N \approx 10$ or $N \approx 12$) or daily periods ($N \approx 60$). Make sure that you use the exact number of days or weeks to the maturity of your option.  
(c) Compare your results to the market price and comment. In particular, discuss the impact of $N$ on your calculated price.  
(d) Price each of the three selected call options using all of (i) approximately monthly periods ($N = 2, 3$), (ii) approximately weekly periods ($N \approx 10, 12$), and (iii) daily periods ($N \approx 60$).  
(e) Compare your results to the market prices and comment. In particular, discuss the impact of the strike price $K$ on your results. In general, once you obtain the parameters $\mu$ and $\sigma$, you should be able to use the computing tool that you developed in HW#3 and in Problem 1 to price your options.


In [48]:
def asset_data(ticker, start_date, end_date):
    '''
    Function Designed to Pull Equity Data
    ----
    ticker - Ticker of desired equity
    start_date -  Date to start pulling data from
    end_data - Date to end data pull, typically today
    '''
    if type(start_date) == datetime.datetime:
        start_date = start_date.strftime('%Y-%m-%d')
    else:
        pass
    
    if type(end_date) == datetime.datetime:
        end_date = end_date.strftime('%Y-%m-%d')
    else:
        pass

    asset = yf.Ticker(ticker)
    data = yf.download(ticker, start_date, end_date)                                # Pull Data
    
    return asset, data

def maturity(timedelta, asset):
    '''
    Function Designed to find option expiry closest to desired expiry
    ----
    timedelta - time until expiry (EXPRESSED IN YEARS)
    asset - yfinance.ticker.Ticker object whos options are being analyzed
    '''
    days = timedelta*365                                                            # Time Expressed in years converted to days
    expiries = asset.options
    distance = []
    for expiry in expiries:                                                         # Calculate Distance from Desired Expiry
        distance.append(abs(datetime.datetime.strptime(expiry, '%Y-%m-%d') - 
                    (datetime.datetime.today() + 
                    datetime.timedelta(days))))

    expiry = expiries[distance.index(min(distance))]                                # Use minimum distance
    return expiry

def rates_data():
    '''
    Function designed to pull current yield on the US 10 year note
    NOTE: Pulls the value as a number, not a percentage
    '''
    rates = yf.download('^tnx', datetime.datetime.today())
    rf = rates['Adj Close'][0] / 100
    return rf

In [49]:
'''Get ETF Data'''
etf, etf_data = asset_data(ticker = 'LQD', start_date='2017-01-01', end_date=datetime.datetime.today())
expiry = maturity(0.25, etf)
px_last = etf_data.iloc[-1,3]

'''Get Option Data'''
options = etf.option_chain(expiry)
calls = options.calls
puts = options.puts
itm_calls = calls[calls['strike'] < px_last - 0.5]
atm_calls = calls[(calls['strike'] < px_last+0.5) & (calls['strike'] > px_last-0.5)]
otm_calls = calls[calls['strike'] > px_last + 0.5]
atm_puts = puts[(puts['strike'] < px_last+0.5) & (puts['strike'] > px_last-0.5)]
atm_k = atm_calls['strike'].to_list()[0]

'''Get Rates Data'''
rf = rates_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Solution to Part (a)

In [62]:
def estimate_params(M, data):
    '''
    Function designed to estimate Mu and Sigma parameters for use in Option Pricing
    ---
    M - Sample Size to consider
    data - Asset DataFrame pulled from Asset_Data() function
    '''

    dt = 1/252                                              # 1 day timestep
    
    y = np.log(1 + data['Adj Close'].pct_change())[-M:]     # Log Returns
    
    sigma_sq = np.var(y) / dt
    sigma = np.sqrt(sigma_sq)                               # Sigma Estimator

    mu = np.mean(y) / dt + 0.5*sigma_sq                     # Mu Estimator
    
    return mu, sigma

In [63]:
lqd_mu, lqd_sigma = estimate_params(M = 500, data = etf_data)
print('The mean estimator is {:.2%}'.format(lqd_mu))
print('The vol estimator is {:.2%}'.format(lqd_sigma))
print('The rates estimator is {:.2%}'.format(rf))

The mean estimator is 4.98%
The vol estimator is 12.11%
The rates estimator is 1.86%


### Solution to Part (b)

In [64]:
lqd_p_tree = build_tree(N=60, T=.25, mu=lqd_mu, sigma=lqd_sigma, s0=px_last, k=atm_k, r=rf, option_type='p', nature='american')
lqd_p_tree.iloc[0,:]

C:\Users\jdean\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


layer                    0
stock_price     122.139999
phi              -0.458849
psi              58.688694
option_value      2.644891
Name: 0, dtype: object

In [61]:
atm_puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
16,LQD220617P00122000,2022-02-28 16:19:31+00:00,122.0,2.02,4.0,4.55,0.0,0.0,20.0,3769,0.159249,True,REGULAR,USD


### Solution to Part (c)

In [13]:
'''Code Block'''

'Code Block'

### Solution to Part (d)

In [14]:
atm_calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
1,LQD220617C00122000,2022-02-15 16:35:27+00:00,122.0,3.3,2.03,2.67,0.0,0.0,NaN,3,0.121347,False,REGULAR,USD


### Solution to Part (e)

In [15]:
'''Code Block'''

'Code Block'

## Problem 3

We consider an N-period binomial model with the following properties: each period lasts a year and the current stock price is S0 = 4. On each period, the stock price doubles when it moves up and is reduced by half when it moves down. The annual interest rate on the money market is $25\%$.

(a) Find the values of $\mu$ and $\sigma$ associated with this CRR model.  
(b) Assuming that $p = \frac{1}{2}$, find $P(S_T > 6)$ when $N = 3$ and $N = 10$.  
(c) Using the risk-neutral option pricing formula (written as an expectation), calculate the price of a European call with strike price $K = 8$ when $N = 3$ and $N = 10$.


### Solution to Part (a)

In [16]:
years = 5
mu, sigma = walk_params(u=1,d=.5,T=years,N=years)
print('The mean estimator is {:.2%}'.format(mu))
print('The vol estimator is {:.2%}'.format(sigma))

The mean estimator is 0.00%
The vol estimator is 69.31%


### Solution to Part (b)

This is clearly a binomial experiment where $N$ represents the time step.  By properties of the tree, its easy to see that there will be $N+1$ nodes and the median of the nodes will be $4$.  Therefore the probability can be represented in the following terms of a binomial distribution

$$P(S_T > 4) = \sum_{i = \lceil \frac{N}{2} + 1 \rceil}^{N+1}{N+1 \choose i}(\frac{1}{2})^{N+1}$$

We can also see that given the doubling/halving property of the tree, the node above $4$ will always be $16$, therefore $P(S_T>4)=P(S_T>6)=P(S_T\geq16)$
$$\implies P(S_T > 6) = \sum_{i = \lceil \frac{N}{2} + 1 \rceil}^{N+1}{N+1 \choose i}(\frac{1}{2})^{N+1}$$

$N=3$ Case
$$P(S_T > 6) = \sum_{i = 3}^{4}{4 \choose i}(\frac{1}{2})^{4} = 4(\frac{1}{2})^4 + (\frac{1}{2})^4 = \frac{5}{2^4} = 0.3125$$

next
$N=10$ Case

$$P(S_T > 6) = \sum_{i = 6}^{11}{11 \choose i}(\frac{1}{2})^{11} = \frac{1 - {11 \choose 5}(\frac{1}{2})^{11}}{2} = 0.3872$$

### Solution to Part (c)

In [17]:
'''Code Block'''

'Code Block'